# `Zomato Bangalore Restaurants: 01 - Initial Data Exploration & Cleaning`

**Objective:** To perform an initial reconnaissance of the 512MB Zomato dataset. The goals are to:
1.  Understand the basic structure, data types, and memory usage.
2.  Identify the extent of missing data and formulate a cleaning strategy.
3.  Get a first look at the key numerical and categorical features.

In [1]:
# --- 1. CORE LIBRARIES ---
import os
import sys
import warnings

# --- 2. DATA HANDLING & ANALYSIS ---
import pandas as pd
import numpy as np

# --- 3. VISUALIZATION ---
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

# --- 4. NLP ---
import nltk
import spacy

# --- 5. PREPROCESSING & MODELING ---
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PowerTransformer, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
# Add other specific models or metrics as needed, e.g., from sklearn.linear_model import LogisticRegression

# --- 6. ADVANCED MODELING (BOOSTING) ---
import xgboost as xgb
import lightgbm as lgb
import catboost as cat

# --- 7. MLOPS & EXPERIMENTATION ---
import mlflow
import optuna

# --- 8. UTILITIES ---
from loguru import logger
from tqdm.auto import tqdm


# ===================================================================
#                      CONFIGURATION
# ===================================================================

# --- 1. PANDAS SETTINGS ---
# Makes DataFrame outputs look professional
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", 1000)
# Formats floats to 2 decimal places with comma separators (e.g., 1,234.56)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

# --- 2. PLOTTING STYLE ---
# Sets a modern, clean style for all matplotlib and seaborn plots
sns.set_theme(style="whitegrid", palette="deep")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["figure.dpi"] = 100  # Makes plots sharper

# --- 3. LOGURU CONFIGURATION ---
# The "beautiful" and "advanced" logger you wanted
logger.remove()  # Remove the default handler
logger.add(
    sys.stdout,
    colorize=True,
    format=(
        "<green>{time:YYYY-MM-DD HH:mm:ss}</green> | "
        "<level>{level: <8}</level> | "
        "<cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - "
        "<level>{message}</level>"
    ),
)
logger.info("✅ All libraries imported and configurations set successfully!")

2025-09-19 11:15:03 | INFO     | __main__:<module>:71 - ✅ All libraries imported and configurations set successfully!


## Data Reading

In [2]:
df_raw = pd.read_csv("../data/raw/zomato_raw.csv")

In [3]:
df_raw.shape

(51717, 17)

In [4]:
df_raw.head(
    5,
)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [6]:
df_raw.duplicated(subset=[col for col in df_raw.columns if col not in ['url']]).sum()

40

In [7]:
df_raw.describe()

,votes
count,"51,717.00"
mean,283.70
std,803.84
min,0.00
25%,7.00
50%,41.00
75%,198.00
max,"16,832.00"


In [8]:
df_raw.describe(include="object")

,url,address,name,online_order,book_table,rate,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
count,51717,51717,51717,51717,51717,43942,50509,51696,51490,23639,51672,51371,51717,51717,51717,51717
unique,51717,11495,8792,2,2,64,14926,93,93,5271,2723,70,22513,9098,7,30
top,https://www.zomato.com/bangalore/jalsa-banasha...,Delivery Only,Cafe Coffee Day,Yes,No,NEW,080 43334321,BTM,Quick Bites,Biryani,North Indian,300,[],[],Delivery,BTM
freq,1,128,96,30444,45268,2208,216,5124,19132,182,2913,7576,7595,39617,25942,3279


In [9]:
df_raw.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes', 'phone', 'location', 'rest_type', 'dish_liked', 'cuisines', 'approx_cost(for two people)', 'reviews_list', 'menu_item', 'listed_in(type)', 'listed_in(city)'], dtype='object')

In [10]:
for col in df_raw.columns.tolist():
    print(f"The first data of {col}: {df_raw[col][10]}\n")

The first data of url: https://www.zomato.com/bangalore/caf%C3%A9-down-the-alley-banashankari?context=eyJzZSI6eyJlIjpbIjE4NTY1Njc5IiwiMTg1OTMwNDQiLCIxODE0ODk0MSIsMTg0NzA3NzEsIjE4NzE2MDYxIiwiMTg2MzI5NzciLCIxODIwMjk5NyIsIjU5MDkwIiwiMTg1NTkyMTEiLCIxODMyMzYzOSIsIjE4NTc0Mjc3IiwiMTg3MjM0OTQiLCIxODg2NzEyMiIsIjE4Njg4NDg5Il0sInQiOiJDYWZcdTAwZTlzIGFuZCBEZWxpcyBpbiBCYW5hc2hhbmthcmkifX0=

The first data of address: 12,29 Near PES University Back Gate, D'Souza Nagar, Hosakerehalli, 3rd Stage, Banashankari, Bangalore

The first data of name: CafÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Down The Alley

The first data of online_order: Yes

The first data of book_table: No

The first data of rate: 4.1/5

The first data of votes: 402

The first data of phone: 080 26724489
+91 7406048982

The first data of location: Banashankari

The first data of rest_type: Cafe

The first data of dish_liked: Waffles, Pasta, Crispy Chicken, Honey Chilli Chicken, Sandwich, Coffee, Crepe

The first data of cuisines: Cafe

The firs

In [11]:
# Inspect the long-text columns for 3 random rows
for index in df_raw.sample(3).index:
    print(f"--- INSPECTING ROW {index} ---")

    print("\n[ADDRESS]")
    print(df_raw.loc[index, "address"])

    print("\n[REVIEWS_LIST]")
    print(df_raw.loc[index, "reviews_list"])

    print("\n" + "=" * 80 + "\n")

--- INSPECTING ROW 43380 ---

[ADDRESS]
57, Samsung Opera House, Junction of Brigade Road & Residency Road, Brigade Road, Bangalore

[REVIEWS_LIST]
[('Rated 4.0', 'RATED\n  Ã\x83Ã\x83Ã\x82Ã\x83Ã\x83Ã\x82Ã\x82ÃÂ£Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x83Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x96Ã\x83Ã\x83Ã\x82Ã\x83Ã\x83Ã\x82Ã\x82ÃÂ£Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x83Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82ÃÂªÃ\x83Ã\x83Ã\x82Ã\x83Ã\x83Ã\x82Ã\x82ÃÂ£Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x82Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82ÃÂ²Ã\x83Ã\x83Ã\x82Ã\x83Ã\x83Ã\x82Ã\x82ÃÂ£Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x83Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82ÃÂ¼Ã\x83Ã\x83Ã\x82Ã\x83Ã\x83Ã\x82Ã\x82ÃÂ£Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x83Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x89Ã\x83Ã\x83Ã\x82Ã\x83Ã\x83Ã\x82Ã\x82ÃÂ£Ã\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x83Ã\x83Ã\x83Ã\x82Ã\x82Ã\x8

---
## **Initial Column-by-Column Analysis & Cleaning Strategy**

Based on the initial `.info()`, `.head()`, and `.describe()` calls, here is a detailed breakdown of each feature and the proposed cleaning plan.

1.  **`url`, `address`, `phone`:**
    -   **Observation:** These columns contain unique, specific information (URLs, street addresses, phone numbers) that is unlikely to have predictive power for a general model. The location data is better captured in the `location` column.
    -   **Plan:** 🎯 Drop these three columns as they are not useful for modeling.

2.  **`name`:**
    -   **Observation:** This is the restaurant's name. While most are unique, the `.describe()` output shows high frequency for chains like 'Cafe Coffee Day', indicating a "chain" feature might be valuable.
    -   **Plan:** 💡 Keep for now, we also need to fix encoding issues in this also. We can later engineer a feature to identify if a restaurant is part of a chain.

3.  **`online_order`, `book_table`:**
    -   **Observation:** Simple binary categorical features stored as "Yes"/"No" strings.
    -   **Plan:** 🧹 Convert these to binary integers (`1` for Yes, `0` for No) for easier analysis and modeling.

4.  **`rate`:**
    -   **Observation:** This is our primary target variable, but it's in a messy state. It's an `object` type (e.g., "4.1/5"), contains `NaN` values, and has special string values like "NEW".
    -   **Plan:** 🧹 This requires significant cleaning. The steps will be:
        1.  Handle the "NEW" entries (perhaps by converting them to `NaN` or a specific numeric value like 0).
        2.  Remove the "/5" suffix.
        3.  Convert the column to a numeric (float) type.
        4.  Develop a strategy to impute the remaining `NaN` values.

5.  **`votes`:**
    -   **Observation:** A clean `int64` column representing the number of votes. It is highly correlated with the `rate`.
    -   **Plan:** ✅ No cleaning needed. This will be a very important feature.

6.  **`location`, `listed_in(city)`:**
    -   **Observation:** Both are key categorical features representing the restaurant's area. They seem related but might have different levels of granularity. `listed_in(city)` appears to be a broader neighborhood category used by Zomato.
    -   **Plan:** 💡 Investigate the relationship between these two columns. Clean them by checking for and consolidating similar names (e.g., "BTM Layout" vs. "BTM").

7.  **`rest_type`, `listed_in(type)`:**
    -   **Observation:** Similar to the location columns, these represent the type of restaurant. `rest_type` seems more specific, while `listed_in(type)` is a broader Zomato category (e.g., "Delivery," "Dine-out").
    -   **Plan:** 💡 Analyze the relationship and potential redundancy. Clean `rest_type` by grouping rare categories.

8.  **`cuisines`, `dish_liked`:**
    -   **Observation:** These are complex, multi-value text fields. `dish_liked` has over 50% missing data, which is a major issue. `cuisines` is more complete.
    -   **Plan:** 💡 These are prime candidates for NLP and feature engineering. For now, we will clean them. The plan for `dish_liked` is uncertain; we may have to drop it due to the high number of missing values.

9.  **`approx_cost(for two people)`:**
    -   **Observation:** This is a crucial feature but is incorrectly typed as `object` due to commas in the numbers (e.g., "1,200"). It also contains `NaN` values.
    -   **Plan:** 🧹 Clean this by removing commas and converting it to a numeric type. Impute missing values, likely with the median.

10. **`reviews_list`:**
    -   **Observation:** This column contains a list of tuples, with each tuple holding the rating and the full text of a review. This is a goldmine of unstructured text data.
    -   **Plan:** 💡 In a later notebook, we will fix the encoding issues and then parse this column to extract features like review sentiment, review length, and keywords. For now, we'll leave it as is.

11. **`menu_item`:**
    -   **Observation:** The vast majority of entries in this column are empty lists (`[]`).
    -   **Plan:** 🎯 This column appears to have very little useful information. The initial plan is to drop it.